<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/autoencoder5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# we upload, in case that we cant parse logs(a lot of time and ram consumption)

# Mount Google Drive to save datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#copy log file
!cp '/content/drive/MyDrive/zHDFS2/logs_train' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/logs_ntest' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/logs_atest' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/log2index' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/reduced_embeddings' '/content/'


In [4]:
import pandas as pd
import json

import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import os
import numpy as np


# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
with open('/content/reduced_embeddings', 'r') as json_file:
    embeddings = json.load(json_file)

In [19]:
len(embeddings[0])


10

In [6]:
# all unique templates in our train datasets

dataset = 'logs_train'
templates = set()

with open('/content/' + dataset, 'r') as f:
        for row in f:
          for temp in row.split():
            templates.add(temp)

print(sorted(templates))
print('nember of templates : ',len(templates))

['0', '1', '10', '11', '12', '13', '14', '15', '16', '2', '3', '4', '5', '6', '7', '8', '9']
nember of templates :  17


In [7]:
# all unique templates in our test datasets

datasets = ['logs_ntest','logs_atest']
templates = set()

for dataset in datasets:
  with open('/content/' + dataset, 'r') as f:
          for row in f:
            for temp in row.split():
              templates.add(temp)

print(sorted(templates))
print('nember of templates : ',len(templates))

['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '6', '7', '8', '9']
nember of templates :  52


In [49]:

name = 'logs_train'
window_size = 10
num_sessions = 0
inputs = []
#outputs = []

with open('/content/' + name, 'r') as f:
        for row in f:
            num_sessions += 1
            line = [ embeddings[int(i)] for i in row.strip().split()]
            for i in range(len(line) - window_size):
                inputs.append(line[i:i + window_size])
                #outputs.append(line[i + window_size])

print('Number of sessions({}): {}'.format(name, num_sessions))
print('Number of seqs({}): {}'.format(name, len(inputs)))
#dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs))
dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float))

Number of sessions(logs_train): 446578
Number of seqs(logs_train): 4704000


In [52]:
print(inputs[0])

[[-0.3891972303390503, 0.12810246646404266, -0.07853731513023376, -0.10766388475894928, 0.23173417150974274, -0.08228960633277893, 0.017653826624155045, -0.030952107161283493, 0.040218982845544815, 0.030139772221446037], [0.38630372285842896, -0.12673094868659973, -0.11596252024173737, -0.18444840610027313, 0.08815333247184753, -0.02099493145942688, 0.10797911882400513, -0.09289209544658661, -0.05879585072398186, 0.03421926125884056], [-0.3891972303390503, 0.12810246646404266, -0.07853731513023376, -0.10766388475894928, 0.23173417150974274, -0.08228960633277893, 0.017653826624155045, -0.030952107161283493, 0.040218982845544815, 0.030139772221446037], [-0.3891972303390503, 0.12810246646404266, -0.07853731513023376, -0.10766388475894928, 0.23173417150974274, -0.08228960633277893, 0.017653826624155045, -0.030952107161283493, 0.040218982845544815, 0.030139772221446037], [-0.21878856420516968, -0.2136467546224594, 0.04706411808729172, 0.20318950712680817, -0.24529194831848145, 0.02122925035

In [12]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers): #embedding dimensions
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers, bidirectional=True,batch_first=True)


    def forward(self, src):

        #src = [src len, batch size]


        outputs, (hidden, cell) = self.rnn(src)

        #outputs = [batch size,src len, hid dim * n directions]
        #hidden = [batch size,n layers * n directions,  hid dim]
        #cell = [batch size, n layers * n directions, hid dim]

        #outputs are always from the top hidden layer

        return outputs[:,-1:,:]  #outputs[:,-1:,:]=output of the end of blocks which is concatenated of two directions(we use -1: for keeping the dimensions)

class Decoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers):       #input _dim depends on encoder's output
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers, bidirectional=False,batch_first=True)

    def forward(self, src):

        #src = [src len, batch size]
        src = torch.tile(src, (1, 10, 1)) # give the output of encoder to each cell of decoder

        outputs, (hidden, cell) = self.rnn(src)

        #outputs = [batch size,src len, hid dim * n directions]
        #hidden = [batch size,n layers * n directions,  hid dim]
        #cell = [batch size, n layers * n directions, hid dim]

        #outputs are always from the top hidden layer

        return outputs

class Autoencoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src):
        encoded = self.encoder(src)
        decoded = self.decoder(encoded)
        return decoded

In [20]:
encoder = Encoder(10,32,1)
decoder = Decoder(64,10,1)
model = Autoencoder(encoder, decoder)
model

Autoencoder(
  (encoder): Encoder(
    (rnn): LSTM(10, 32, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (rnn): LSTM(64, 10, batch_first=True)
  )
)

In [21]:
input_data = torch.rand(1, 10, 10) #(batch,sequence_length,embedding)
output= model(input_data)
print(output.shape)

torch.Size([1, 10, 10])


In [23]:
# Calculate the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of Parameters: {num_params}")

Number of Parameters: 14304


In [53]:
dataloader = DataLoader(dataset, batch_size=128)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

In [54]:
len(dataloader) # regard to batch size

36750

In [57]:
for step, (seq) in enumerate(dataloader):
  print(seq[0].shape)
  break

torch.Size([128, 10, 10])


In [ ]:

num_epochs = 100
input_size = 10 # embedding size

# Train the model
start_time = time.time()
total_step = len(dataloader)
for epoch in range(num_epochs):  # Loop over the dataset multiple times
    train_loss = 0
    for step, (seq) in enumerate(dataloader):
        # Forward pass
        seq = seq[0].clone().detach().view(-1, window_size, input_size).to(device)
        output = model(seq)
        loss = criterion(output, seq.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step))
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
print('Finished Training')
